In [1]:
import ujson as json
import pandas as pd

#data_file = 'graphnodes.ndjson'
data_file = 'testdata/graphnodes_edgecase_node_9.ndjson'
records = map(json.loads, open(data_file))
df = pd.DataFrame.from_records(records)

FileNotFoundError: [Errno 2] No such file or directory: 'graphnodes_edgecase_node_9.ndjson'

In [ ]:
neighbor_nodes = []
for col1, row in zip(df["node_id"], df["node"]):
    for d in row:
        for k, v in d.items():
            neighbor_nodes.append({"node_id": col1, "neighbor_nodes": k, "direction": v})
df_1 = pd.DataFrame(neighbor_nodes)
#relevant_nodes_list = tmp["node_id"].unique(). 

In [ ]:
transaction_time = []
transaction_list = []
for col1, col2 in zip(df["transaction_val"], df["transaction_time"]):
    for c1 in col1:
        transaction_list.append(c1)
    for c2 in col2:
        transaction_time.append(c2)
df_1["transaction_val"] = transaction_list
df_1["transaction_time"] = transaction_time

df["FA_1_case_1"] = df["labels"].apply(lambda x: x["FA_1_case_1"])
df["FA_1_case_2"] = df["labels"].apply(lambda x: x["FA_1_case_2"]) 

df_final = pd.merge(df_1, df[["node_id","FA_1_case_1", "FA_1_case_2"]], on="node_id", how="left")

In [ ]:
# filter out single transation
tmp = (df_final.groupby(["node_id","direction"]).count() == 1).neighbor_nodes.reset_index().rename(columns={"neighbor_nodes":"single_trans"})
tmp = tmp.loc[tmp.single_trans == False].drop(["single_trans"], axis=1)

# create the df_mask dataframe
# mask_1_case_1 checks if it's a "in" relation 
# mask_1_case_2 checks if it's a "out" relation

df_mask = pd.merge(tmp,df_final, on = ["node_id","direction"], how="left")
df_mask["mask_1_case_1"] = df_mask["direction"].apply(lambda x: True if x == "out" else False)
df_mask["mask_1_case_2"] = df_mask["direction"].apply(lambda x: True if x == "in" else False)

# filter out the trans_val > 10000
# mask_2_case_1 checks if sum of all "in" transactions is > 10000
# mask_2_case_2 checks if sum of all "out" transactions is > 10000
df_mask = df_mask[df_mask["transaction_val"] < 10000]

tmp_2 = df_mask.groupby(["node_id","direction"]).sum().transaction_val.reset_index()
tmp_2["threshold_tot_trans_val"] = tmp_2["transaction_val"] > 10000
df_mask = pd.merge(df_mask, tmp_2.drop(["transaction_val"],axis=1), on = ["node_id","direction"], how="left")

df_mask["mask_2_case_1"] = df_mask["mask_1_case_1"] & df_mask["threshold_tot_trans_val"]
df_mask["mask_2_case_2"] = df_mask["mask_1_case_2"] & df_mask["threshold_tot_trans_val"]
#df_mask

In [ ]:
case1_candidates = df_mask[df_mask['mask_2_case_1']]
case2_candidates = df_mask[df_mask['mask_2_case_2']]
case2_candidates

In [ ]:
#no time-jump, no self-call
def thresholds_FA1(df_node, start_index):
    node_flag = False
    end = df_node.shape[0]
    total_amount = 0
    #Sum over amounts until Threshold is reached
    for i in list(range(start_index, end)):
        total_amount = total_amount + df_node['transaction_val'].values[i]
        if total_amount >= 10000:
            time_window = df_node['transaction_time'].values[i] - df_node['transaction_time'].values[start_index]
            #When Threshold is reached, check if it is in defined time windowd (48h = 172800s)
            if time_window <= 172800:
                node_flag = True
                return node_flag
            else:
                i = start_index + 1
                start_index = i
                #node_flag = thresholds_FA1(df_node, start_index)
    return node_flag

In [ ]:
def flagger_FA1(df_candidates):
    start_index = 0
    #j keeps track of current line in df_candidates to check if next node is same as previous
    #j = 0
    node_flags = {}
    #loop over all candidates
    for node in df_candidates['node_id'].unique().tolist():
        df_node = df_candidates[df_candidates['node_id'] == node].sort_values(by='transaction_time').reset_index(drop=True)
        node_flag = thresholds_FA1(df_node, start_index)
        node_flags[node] = node_flag
    return node_flags

In [ ]:
case1_flags = flagger_FA1(case1_candidates)
case2_flags = flagger_FA1(case2_candidates)
print([case1_flags, case2_flags])

In [ ]:
#time jump, self-call
def thresholds_FA1_time_jump(df_node, start_index):
    node_flag = False
    end_index = df_node.shape[0]
    time_window_end = df_node['transaction_time'].values[start_index] + 172801
    index_latest_trx = df_node['transaction_time'].searchsorted(time_window_end, side = 'left') - 1
    trx_sum = df_node['transaction_val'].iloc[start_index:index_latest_trx + 1].sum()
    if trx_sum >= 10000:
        node_flag = True
    else:
        start_index = start_index + 1
        if start_index == end_index:
            node_flag = False
        else:
            node_flag = thresholds_FA1_time_jump(df_node, start_index)
    return node_flag

In [ ]:
def flagger_FA1_time_jump(df_candidates):
    start_index = 0
    #j keeps track of current line in df_candidates to check if next node is same as previous
    #j = 0
    node_flags = {}
    #loop over all candidates
    for node in df_candidates['node_id'].unique().tolist():
        df_node = df_candidates[df_candidates['node_id'] == node].sort_values(by='transaction_time').reset_index(drop=True)
        node_flag = thresholds_FA1_time_jump(df_node, start_index)
        node_flags[node] = node_flag
    return node_flags

In [ ]:
case1_flags = flagger_FA1_time_jump(case1_candidates)
case2_flags = flagger_FA1_time_jump(case2_candidates)
print([case1_flags, case2_flags])

In [ ]:
# time-jump, no self-call
def nr_thresholds_FA1_time_jump(df_node, start_index):
    end_index = df_node.shape[0]
    for i in  list(range(start_index, end_index)):
        node_flag = False
        time_window_end = df_node['transaction_time'].values[start_index] + 172801
        index_latest_trx = df_node['transaction_time'].searchsorted(time_window_end, side = 'left') - 1
        trx_sum = df_node['transaction_val'].iloc[start_index:index_latest_trx + 1].sum()
        if trx_sum >= 10000:
            node_flag = True
        else:
            start_index = start_index + 1
            if start_index == end_index:
                node_flag = False
    return node_flag

In [ ]:
def nr_flagger_FA1_time_jump(df_candidates):
    start_index = 0
    #j keeps track of current line in df_candidates to check if next node is same as previous
    #j = 0
    node_flags = {}
    #loop over all candidates
    for node in df_candidates['node_id'].unique().tolist():
        df_node = df_candidates[df_candidates['node_id'] == node].sort_values(by='transaction_time').reset_index(drop=True)
        node_flag = nr_thresholds_FA1_time_jump(df_node, start_index)
        node_flags[node] = node_flag
    return node_flags

In [ ]:
case1_flags = nr_flagger_FA1_time_jump(case1_candidates)
case2_flags = nr_flagger_FA1_time_jump(case2_candidates)
print([case1_flags, case2_flags])

In [ ]:
dummy_flags = [False] * df_final.shape[0]
df_final['Case1_Flag'] = dummy_flags
df_final['Case2_Flag'] = dummy_flags
for node in df_final['node_id']:
    #mask = (int(df_final['node_id']) == node)
    #print(node)
    if node in case1_flags:
        df_final.loc[(df_final['node_id'] == node, 'Case1_Flag')] = case1_flags[node]
    if node in case2_flags:
        df_final.loc[(df_final['node_id'] == node, 'Case2_Flag')] = case2_flags[node]

In [ ]:
df_final